In [1]:
import numpy as np
from tqdm import tqdm
import requests
import pandas as pd
import matplotlib.pyplot as plt
import json
from datetime import datetime
from sklearn.linear_model import LinearRegression
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3
plt.rcParams['axes.titlesize'] = 18

In [2]:
lg = pd.read_csv('logs-6097258.csv')
lg.head()

,insertId,jsonPayload.blockHash,jsonPayload.builderEntry,jsonPayload.builderPubkey,jsonPayload.contentLength,jsonPayload.duration,jsonPayload.error,"jsonPayload.""gzip-req""",jsonPayload.level,jsonPayload.method,...,receivedLocation,resource.labels.cluster_name,resource.labels.container_name,resource.labels.location,resource.labels.namespace_name,resource.labels.pod_name,resource.labels.project_id,resource.type,severity,timestamp
0,xgh77v0tplpzq928,0x1be4076fcdf6692bcc25476c9fff68da0f5645ab0db8...,NaN,NaN,NaN,NaN,NaN,NaN,info,getHeader,...,NaN,midori,proposer-api,europe-west1-b,default,proposer-api-78c4486fb8-qblzz,ultra-sound-relay,k8s_container,INFO,2023-03-28T08:12:09Z
1,mv3ixuee7li7nsw6,0x1be4076fcdf6692bcc25476c9fff68da0f5645ab0db8...,NaN,NaN,NaN,NaN,NaN,NaN,info,getHeader,...,NaN,midori,proposer-api,europe-west1-b,default,proposer-api-78c4486fb8-qblzz,ultra-sound-relay,k8s_container,INFO,2023-03-28T08:12:09Z
2,b0d6osjd1pc7p8by,0x1be4076fcdf6692bcc25476c9fff68da0f5645ab0db8...,NaN,NaN,NaN,NaN,NaN,NaN,info,getHeader,...,NaN,midori,proposer-api,europe-west1-b,default,proposer-api-78c4486fb8-qblzz,ultra-sound-relay,k8s_container,INFO,2023-03-28T08:12:09Z
3,46erkii6l8nxz46v,0x1be4076fcdf6692bcc25476c9fff68da0f5645ab0db8...,NaN,NaN,NaN,NaN,NaN,NaN,info,getHeader,...,NaN,midori,proposer-api,europe-west1-b,default,proposer-api-78c4486fb8-xxbhn,ultra-sound-relay,k8s_container,INFO,2023-03-28T08:12:09Z
4,1xkptwdgeis2uylt,0x1be4076fcdf6692bcc25476c9fff68da0f5645ab0db8...,NaN,NaN,NaN,NaN,NaN,NaN,info,getHeader,...,NaN,midori,proposer-api,europe-west1-b,default,proposer-api-78c4486fb8-xxbhn,ultra-sound-relay,k8s_container,INFO,2023-03-28T08:12:09Z


In [6]:
lg.keys()

Index(['insertId', 'jsonPayload.blockHash', 'jsonPayload.builderEntry',
       'jsonPayload.builderPubkey', 'jsonPayload.contentLength',
       'jsonPayload.duration', 'jsonPayload.error', 'jsonPayload."gzip-req"',
       'jsonPayload.level', 'jsonPayload.method', 'jsonPayload.mevBoostV',
       'jsonPayload.msg', 'jsonPayload.numWaiting',
       'jsonPayload.optBlocksInFlight', 'jsonPayload.parentHash',
       'jsonPayload.profile', 'jsonPayload.proposerPubkey',
       'jsonPayload.pubkey', 'jsonPayload.service', 'jsonPayload.slot',
       'jsonPayload.tx', 'jsonPayload.ua', 'jsonPayload.value',
       'jsonPayload.version', 'labels."compute.googleapis.com/resource_name"',
       'labels."k8s-pod/app"', 'labels."k8s-pod/pod-template-hash"',
       'labels."k8s-pod/project"', 'logName', 'receiveLocation',
       'receiveTimestamp', 'receivedLocation', 'resource.labels.cluster_name',
       'resource.labels.container_name', 'resource.labels.location',
       'resource.labels.namespace_n

In [11]:
bu = lg[(lg['labels."k8s-pod/app"']=='builder-api') & (lg['jsonPayload.method']=='submitNewBlock')]
bu['timestamp'] = pd.to_datetime(bu['timestamp'], format="%Y-%m-%dT%H:%M:%SZ")
bu.head()

<ipython-input-11-627953695e6a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bu['timestamp'] = pd.to_datetime(bu['timestamp'], format="%Y-%m-%dT%H:%M:%SZ")


,insertId,jsonPayload.blockHash,jsonPayload.builderEntry,jsonPayload.builderPubkey,jsonPayload.contentLength,jsonPayload.duration,jsonPayload.error,"jsonPayload.""gzip-req""",jsonPayload.level,jsonPayload.method,...,receivedLocation,resource.labels.cluster_name,resource.labels.container_name,resource.labels.location,resource.labels.namespace_name,resource.labels.pod_name,resource.labels.project_id,resource.type,severity,timestamp
24,u5e69ev5f6t3b4tb,0xa50384243959534c0df2d48b92df118beacb0a7c4c40...,NaN,0x96a59d355b1f65e270b29981dd113625732539e955a1...,114145.0,0.147590,NaN,NaN,info,submitNewBlock,...,NaN,midori,builder-api,europe-west1-b,default,builder-api-855d777575-qg8hk,ultra-sound-relay,k8s_container,INFO,2023-03-28 08:12:00
38,i998nzon8hg70j5l,0xa50384243959534c0df2d48b92df118beacb0a7c4c40...,NaN,0x96a59d355b1f65e270b29981dd113625732539e955a1...,114145.0,NaN,NaN,NaN,info,submitNewBlock,...,NaN,midori,builder-api,europe-west1-b,default,builder-api-855d777575-qg8hk,ultra-sound-relay,k8s_container,INFO,2023-03-28 08:12:00
39,i80p2e8xqreobv0j,0xc4ceebd39a0edf295eb1fdebe703acaf14b997350d1b...,NaN,0xb9b50821ec5f01bb19ec75e0f22264fa9369436544b6...,354748.0,0.026027,simulation failed: unknown ancestor,NaN,error,submitNewBlock,...,NaN,midori,builder-api,europe-west1-b,default,builder-api-855d777575-tfd4b,ultra-sound-relay,k8s_container,ERROR,2023-03-28 08:12:00
47,bty6xr0a8p12kpsl,0xc32f801823fe4bff466f9b76bd013aadb360d08a6df8...,NaN,0x8efc1675ffb449abc00a6ad8a2808cdf798d96fbb979...,356522.0,0.139426,NaN,NaN,info,submitNewBlock,...,NaN,midori,builder-api,europe-west1-b,default,builder-api-855d777575-l79gt,ultra-sound-relay,k8s_container,INFO,2023-03-28 08:12:00
59,41n6t22ly4cb5hq7,0x1be4076fcdf6692bcc25476c9fff68da0f5645ab0db8...,NaN,0xa971c4ee4ac5d47e0fb9e16be05981bfe51458f14c06...,382372.0,NaN,NaN,NaN,info,submitNewBlock,...,NaN,midori,builder-api,europe-west1-b,default,builder-api-855d777575-qg8hk,ultra-sound-relay,k8s_container,INFO,2023-03-28 08:12:00


In [21]:
bu['jsonPayload.error']

24                                     NaN
38                                     NaN
39     simulation failed: unknown ancestor
47                                     NaN
59                                     NaN
                      ...                 
878                                    NaN
879                                    NaN
883                                    NaN
884                                    NaN
885                                    NaN
Name: jsonPayload.error, Length: 558, dtype: object

In [22]:
errs = bu[bu['jsonPayload.error'] == 'simulation failed: unknown ancestor']
len(errs)

235

In [23]:
errs['timestamp'].value_counts()

2023-03-28 08:11:58    107
2023-03-28 08:11:59     64
2023-03-28 08:11:57     63
2023-03-28 08:12:00      1
Name: timestamp, dtype: int64

In [26]:
errs['jsonPayload.parentHash'].value_counts()

0xff540a481885da0916af298748638eb6e5e72ad24fed69ffe93cb434bc690dcd    235
Name: jsonPayload.parentHash, dtype: int64

In [24]:
sucs = bu[bu['jsonPayload.msg'] == 'block validation successful']
len(sucs)

112

In [25]:
sucs['timestamp'].value_counts()

2023-03-28 08:11:58    50
2023-03-28 08:11:59    38
2023-03-28 08:11:57    21
2023-03-28 08:12:00     3
Name: timestamp, dtype: int64

In [27]:
sucs['jsonPayload.parentHash'].value_counts()

0xff540a481885da0916af298748638eb6e5e72ad24fed69ffe93cb434bc690dcd    112
Name: jsonPayload.parentHash, dtype: int64

In [ ]:
sucs[]

In [10]:
len(bu)

558